## **1. Regroupement en sous-corpus**
Plutôt que de récupérer un tag qui aurait été assigné manuellement pour chaque lien, on va le créer en déconstruisant les URLs et en créant des sous-dossiers correspondant à différents sous-niveaux dans l'arborescence du site.  

À décider : jusqu'à quel niveau on regroupe (ça peut être un peu du cas par cas selon l'organisation du site)

Créer des sous-dossiers à l'intérieur

In [ ]:
path = '../03-corpus/0-listes_sites_html_pdf/'
acteur = "chu_iu"
sous_acteur = 'chu_qc'

from pandas import *

# encoding= 'ISO-8859-1' "utf-8"
with open(path + acteur  + '.csv', encoding='ISO-8859-1') as f:
    csv = read_csv(f, sep=';')

In [ ]:
csv

In [ ]:
liste = [lien for lien in csv['Address'].tolist() if not 'actualites' in lien and not 'propos' in lien and not 'joindre' in lien and not 'communiques' in lien and not 'about' in lien]
liste = [lien for lien in liste if 'chudequebec.ca' in lien]

In [ ]:
"""
0 = http/s
1 = //
2 = domaine principal (ex. INSPQ)
3 = 1e sous-niveau
4 = 2e sous-niveau
5 = 3e sous-niveau
etc.
"""

liens_2N = [lien for lien in liste if len(lien.split('/')) >= 3]
liens_3N = [lien for lien in liste if len(lien.split('/')) >= 4]
liens_4N = [lien for lien in liste if len(lien.split('/')) >= 5]

In [ ]:
lien = liste[20]
lien.split('/')

In [ ]:
regroupements = {}

for lien in liens_2N:
    tag = lien.split('/')[3]
    if tag in regroupements:
        regroupements[tag].append(lien)
        
    else:
        if len(tag)>1:
            regroupements[tag] = []

tags = list(regroupements.keys())

In [ ]:
# On garde seulement les sections qui contiennent plus de 15 sous-pages
tags_retenus = [tag for tag in tags if len(regroupements[tag]) >10]

In [ ]:
print('Ça nous donne {} tags.'.format(len(tags_retenus)))
tags_retenus

In [ ]:
import os, shutil
from pathlib import Path

# Créer un sous-dossier où on va aller copier les fichiers .txt correspondant au sous-corpus
path='../03-corpus/2-sous-corpus/' + acteur + '/'

Path(path).mkdir(parents=True, exist_ok=True)


for tag in tags_retenus:
    subfolder =  tag + '/' #le nom du tag

    Path(path+subfolder).mkdir(parents=True, exist_ok=True)

    # Aller chercher les fichiers .txt correspondant au sous-corpus dans le corpus principal et les copier dans le sous-dossier qu'on vient de créer
    src = '../03-corpus/2-data/1-fr/' + acteur + '/' 
    dst = path + subfolder

    for file in os.listdir(src):
        if (file.endswith(".txt") and not "corpus" in file):
            file_path = src + file
            
            with open(file_path, 'r', encoding = "UTF-8") as f:
                data = f.readlines()
                try: 
                    url = data[0]
                    if(tag in url):
                        shutil.copy(src + file, dst)
                except:
                    True
                    

In [ ]:
path='../03-corpus/2-sous-corpus/' + acteur + '/'
liste_total = sum([len(files) for r, d, files in os.walk(path)])

In [ ]:
path='../03-corpus/2-sous-corpus/' + acteur + '/'

liste_fichiers = []
for subfolder in os.listdir(path):
    liste_fichiers += os.listdir(path + subfolder)

liste_fichiers = set(liste_fichiers)

In [ ]:
len_corpus_global = len(os.listdir(src))
print('Pages dans le corpus global : ' + str(len_corpus_global))

os.chdir('/Users/camilledemers/Documents/03-corpus/2-sous-corpus/' + acteur + '/')
print('Pages (total) dans les sous-corpus : ' + str(liste_total) + ' (certaines pages peuvent avoir plus d\'un tag et donc apparaître deux fois).')
print('Pages (uniques) dans les sous-corpus : ' + str(len(liste_fichiers)))
print('\nIl y a donc {} pages de notre corpus global qui ne se retrouvent pas dans le sous-corpus'.format(str(len_corpus_global - len(liste_fichiers))))


À partir d'ici, on peut aller analyser les sous-corpus constitués en changeant le path et le 'acteur' dans le Notebook *Prétraitement.ipynb*  (on pourrait d'ailleurs changer le nom de la variable 'acteur' pour indiquer plutôt le critère/thème de regroupement de manière générale)